# 2223-MA378 : Lab 2
## Lab 2 of Numerical Analysis 2: Experiments with spline interpolation.
* *Name*: Niall Madden [Change this to your name before submitting!!!!] 
* *ID*: 01234567 [Change this to your ID]
* *Email*:  `n.madden321@universityofgalway.ie` [Change this to your email address]
* *Date*: Week 8, 2023/2024
* *Collaborators*: List anyone you worked with on this

***

* Connect to [https://cloudjupyter.universityofgalway.ie/] and log in with the details provided to you in an email from `maths-stomaths-sto@universityofgalway.ie`
* Upload the notebook from https://www.niallmadden.ie/2324-MA378/#lab2
* Upload your solutions to Exercises 0, 1, 2 and 3 Canvas (2324-MA378... Assaignments... Lab 2) by 5pm, Friday 1st March.
* Ideally, upload your solutions as a `.pdf` file. Failing that, try `.html` or `.ipynb`. 

***

***

## Collaboration policy
Collaboration is encouraged. If you work with some class-mates, please list their names in the first cell. However, every member of the team has to upload their own version.

## The Octave Primer
We are using Octave, on a Jupyter Server, for this lab. 

For more on getting started with Octave, please work through the primer at [https://www.niallmadden.ie/2324-MA378/OctavePrimer.html]

Due to a minor bug in Octave, you'll see a message: `warning: using the gnuplot graphics toolkit is discouraged`. You can safely ignore that.

***

## Exercise 0
Change the data at the start of this file to include your name, ID number and email address.

## Outline
In class, we did a few examples of writing out piecewise polynomial interpolants of functions. However, in practice they are always implemented by computer. In this lab, we  will look at how this is done in Octave/MATLAB. Moreover, you will investigate how the numerical results compare with the theory. This has two purposes. 

1. To verify that the theory is correct. This is of particular interest for the error estimate given for cubic splines since they  were presented without proof.
2. More important) to verify that the theory is sharp. For example, we might prove that the error for piecewise linear interpolation is bounded by $Ch^2$, for some constant, $C$. However, since $h \leq 1$, this does not exclude the possibility that in fact it can be bounded by $Ch^3$. However this is easily verified experimentally.

## Some Spline  Basics
Let $\{x_i\}_{i=0}^N := \{x_0 < x_1 < \dots < x_N\}$ be a set of interpolation points.  
The piecewise linear interpolant, $l$, of a function  $f$, at those points is defined as follows 
1. $l$ is continuous on $[x_{0},x_N]$,
2. On each interval $[x_{i-1}, x_i]$, $l(x)=l_i(x)$ where $l_i$ is a polynomial of degree 1, for $i=1, 2, \dots, N$.
3. $l(x_i) = f(x_i)$ for each $i=0, 1, \dots, $N$.

Review your notes from Section 2.1 to see how to write down a formula for the $l_i$, 
    
We also proved that $\| f - l \|_\infty \leq \frac{h^2}{8}\|f''\|_\infty$   where  $h = x_{i} - x_{i-1}$, 
and $\displaystyle \| u\|_\infty := \max_{x_0 \leq x \leq x_N} |u(x)|$.

## Computing Piecewise Polynomial Interpolants

In Octave/MATLAB, the function `interp1()` computes various types of piecewise polynomial interpolants to a data set, one one dimension. The syntax for computing the piecewise linear interpolant to the 
data set $\{(x(1), y(1)), (x(2), y(2)), ... (x_N, y_N)\}$ is:

where `x` is a vector of strictly increasing points, and `y` is a vector of the same length as `x`. 
The function returns a structure representing a piecewise polynomial. It can then be evaluated using the `ppval()` function.

### Example
Let's compute the piecewise linear interpolant to the points $\{ (0,0), (1,1), (2,1), (3,6), (4,5)\}$.

In [ ]:
x = [0, 1, 2, 3, 4]; % interpolation points
y = [0, 1, 1, 6, 5]; % interpolation values
l = interp1(x,y,'linear', 'pp');
xp = x(1):0.01:x(end) ; % points at which we'll plot l.

plot(x,y, 'o', xp, ppval(l, xp));
legend('Interpolation data', 'PW Linear interpolant', 'Location','northwest')

***

## Interpolating a function
Let's interpolate $f(x)=\sin(\pi x)$ on $[0,1]$ with $N=4$. 

In [ ]:
f = @(x)sin(pi*x);
N = 4;
h = 1/N;
x = 0:h:1; % Interpolation points
y = f(x);  % Interpolation values
l = interp1(x, y, 'linear', 'pp'); % Compute the interpolant

Plot the interpolant and the error

In [ ]:
xp = 0:h/10:1; % points for plotting.

subplot(2,1,1)
plot(xp, f(xp), x, y, 'o', xp, ppval(l, xp));
legend('f(x_i)', 'f(x)', 'l(x)')
title('f(x) and l(x)');

subplot(2,1,2)
plot(xp, abs(f(xp)-ppval(l, xp)), x, x*0, 'o');
title('|f(x) - l(x)|');

Display the error:

In [ ]:
error=max(abs(f(xp)-ppval(l, xp)));
fprintf('N=%4d, h=%8.2e, ||f-l||=%9.3e\n', N , h, error); 

***

## Verifying Convergence
We'll verify that the method converges at a rate that is proportional to $h^2$. For that, we'll use a for loop:

In [ ]:
k=0;
for N=[8, 16, 32, 64, 128, 256, 512]; % the values of $N$ we'll use
   k=k+1;
   h = 1/N;
   Hs(k)=h; % keep track of the values of h used
   x = 0:h:1;
   l = interp1(x, f(x), 'linear', 'pp');
   xp = 0:h/10:1;
   Errors(k)=max(abs(f(xp)-ppval(l, xp)));
   fprintf('N=%4d, h=%8.2e, ||f-l||=%9.3e\n', N , h, Errors(k));
end

Create a log-log plot of both the errors, and $h^2$ plotted against $h$:

In [ ]:
loglog(Hs, Errors, ':o', Hs, Hs.^(2), '--');
legend('Errors', 'h^{2}')

You should observe that, in this log-log plot, the lines representing the errors and $h^{2}$ appear to be parallel. 
This implies that, indeed, the error is proportional to $h^2$. 


***

## EXERCISE 1 : Find $C$ for linear splines
We expect that the error is (approximately) $Ch^2$. Use the data above to estimate $C$. Use that value of $C$ to determine the value of $N$ you'd need to take to ensure that the error is no more than $10^{-12}$. Show your calcuations in a cell below. 

**YOUR CODE GOES HERE:**

In [ ]:
## Insert here your code for 
## 1. estimating C 
## 2. using that to find the N for which the error is at most 1e-12.


***

## Cubic Spline Interpolant
The `spline()` function computes the natural cubic spline interpolant to a function. The syntax is 

### EXERCISE 2: Verify convergence for cubic splines

Taking $f(x)=\sin(\pi x)$, $x_0=0$ and, $x_N=1$, we want to verify that $\| f - S \|_\infty \leq C h^4$, and estimate the value of $C$. 
Do this by producing a log-log plot of $\|f-S\|_\infty$ against various values of $h$. Again, find the value of $N$ required so that the error is at most $10^{12}$.

Specifically:

1. Copy the code from above that computes the linear spline interpolant to $f(x)=\sin(\pi x)$ for $N=8, 16, \dots, 512$ 
2. Modify the code so that it computers $S$ instead of $l$
3. Copy the code for generating the log-log plot, but adjust it so that it plots $S$ (rather than $l$) and compares this with `Hs.^(4)` rather then `Hs.^(2)`.
4. Compute an estimate for $C$, and from that the value of $N$ needed to ensure the error is less than $10^{-12}$.

In [ ]:
## Insert here your code for  Exercise 2 here 



***

## How accurate is PCHIP interpolation?

### EXERCISE 3: Determine the order of accuracy of the `pchip` method.

Easlier we verified the order of convergence of the _linear_ and _spline_ methods, which are implemeted as the `interp1()` and `spline()` functions, respectively. 

Now we want to determine the order of accuracy of the Piecewise Cubic Hermite Interpolating Polynomial (_pchip_) method, which is implemented in Octave as `pchip()`. 
However, it is not exactly the same as the methos in Section 2.3, for which we had to provide not just $f(x)$, but also $f'(x)$. In the `pchip()` function the algorithm computes $f'$ numerically. Therefore, although the theory suggests that the error is proportional to $h^{-4}$, that is not what we get.

**Exercise**: If the error for the `pchip()` method is proportional to $h^p$, determine $p$.

#### SOLUTION

In [ ]:
## Insert here your code for Exercise 3 here 


We can see that the slope of the error line most resembles the slope of $h^2$. To we expect that error is bounded by $Ch^2$